In [ ]:
!pip install scikit-plot
!pip install jax-unirep

Try to deal with GPU memory 
accroading to https://jax.readthedocs.io/en/latest/gpu_memory_allocation.html

In [1]:
import tensorflow as tf
print("Num of GPUs available: ", len(tf.test.gpu_device_name()))

Num of GPUs available:  13


In [1]:
import os

os.environ['XLA_PYTHON_CLIENT_PREALLOCATE']='False'
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION']='.2'
os.environ['XLA_PYTHON_CLIENT_ALLOCATOR']='platform'

In [2]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt

from Bio import SeqIO
from jax_unirep import get_reps
from jax_unirep import evotune, fit
from jax_unirep.utils import dump_params

In [3]:
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)

gpu


In [4]:
fastas = "/home/ubuntu/data/bk_fasta/SRR5868581_2.len20.assembly.fasta"
#plk = "/mnt/vdb/phase3/TaiwaneseOolong.len20.pkl"

In [5]:
with open(fastas) as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    seqs = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        # Remove leading and trailing characters from a string
        seqs.append(str(seq_record.seq.strip('*')))
        lengths.append(len(seq_record.seq))

In [6]:
# dictionary of lists  
dict = {'ID': identifiers, 'Sequence': seqs, 'length': lengths}  
df = pd.DataFrame(dict) 
#df["Sequence"] =  seqs
#df.to_pickle(plk)    
df  

,ID,Sequence,length
0,0,GRWNNAGKSEEHTSELQSRSDLVCRLLLEKKRGGGG,36
1,1,MHIGKIEEHTSELQSRSDLVCRHQLEKKNGGG,32
2,2,DVCSSDYPGFRFIPHRQVCLPKMAHLELSIPWRGSTEQPRRP,42
3,3,TTLGQTEEHTSELQSRSDLVCRLQLEKKKGG,31
4,4,YDGYPSSNTFKLTLTDLMSHSQFHSLN,28
...,...,...,...
1058124,1058124,AGHTEQIGRAHWARSEEHTGQDRKRTQ,28
1058125,1058125,SEEHTSELQSRSYLVSRLLLEKKKEG,26
1058126,1058126,SSLIASSAYQKWHTWSSRFRGAAQRSSRAVLPI,34
1058127,1058127,ASPVLLTKNGTLGALDSVARLNGAAAPSYLFKV,34


Remove Duplicate

In [7]:
df.drop_duplicates(subset=['Sequence'],inplace=True)
df

,ID,Sequence,length
0,0,GRWNNAGKSEEHTSELQSRSDLVCRLLLEKKRGGGG,36
1,1,MHIGKIEEHTSELQSRSDLVCRHQLEKKNGGG,32
2,2,DVCSSDYPGFRFIPHRQVCLPKMAHLELSIPWRGSTEQPRRP,42
3,3,TTLGQTEEHTSELQSRSDLVCRLQLEKKKGG,31
4,4,YDGYPSSNTFKLTLTDLMSHSQFHSLN,28
...,...,...,...
1058124,1058124,AGHTEQIGRAHWARSEEHTGQDRKRTQ,28
1058125,1058125,SEEHTSELQSRSYLVSRLLLEKKKEG,26
1058126,1058126,SSLIASSAYQKWHTWSSRFRGAAQRSSRAVLPI,34
1058127,1058127,ASPVLLTKNGTLGALDSVARLNGAAAPSYLFKV,34


In [8]:
#0-200000
#200000-579590
#579590-1159181
#df1 = df.iloc[0:100000, :]
#df1 = df.iloc[100000:200000, :]
# df1 = df.iloc[200000:300000, :]
# df1 = df.iloc[300000:400000, :]
# df1 = df.iloc[400000:500000, :]
# df1 = df.iloc[500000:600000, :]
# df1 = df.iloc[600000:700000, :]
# df1 = df.iloc[700000:800000, :]
# df1 = df.iloc[800000:900000, :]
# df1 = df.iloc[900000:1000000, :]
df1 = df.iloc[1000000:1159181, :]

#df2 = df.iloc[:, :]
df1

,ID,Sequence,length
1007174,1007174,PVEGGGPVATEGGDERDSPQSRVAWECSPKR,31
1007175,1007175,PLRAALPSNPTLRRVPLVAALRRYGPPTL,29
1007176,1007176,RRGLFRSWGCLLLKSNGGARRLANPGRTSGG,31
1007177,1007177,HRSFQYQMMPLNILWGTNLPFERLSP,27
1007178,1007178,NRDSWGHPYLIVRGEILGSIKDELMR,26
...,...,...,...
1058124,1058124,AGHTEQIGRAHWARSEEHTGQDRKRTQ,28
1058125,1058125,SEEHTSELQSRSYLVSRLLLEKKKEG,26
1058126,1058126,SSLIASSAYQKWHTWSSRFRGAAQRSSRAVLPI,34
1058127,1058127,ASPVLLTKNGTLGALDSVARLNGAAAPSYLFKV,34


In [9]:
_h_avg, h_final, c_final= get_reps(df1['Sequence'].to_list())

In [10]:
df1.drop(columns=['Sequence', 'length'], inplace=True)
df1['reps']=_h_avg.tolist()
df1.to_pickle("/mnt/vdb/phase3/TaiwaneseOolong.len20.1000000_1159181.pkl")
df1

/home/ubuntu/miniconda3/envs/py3/lib/python3.7/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/ubuntu/miniconda3/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ID,reps
1007174,1007174,"[0.023784521967172623, -0.08420483767986298, -..."
1007175,1007175,"[0.023177096620202065, -0.027476267889142036, ..."
1007176,1007176,"[0.0200453232973814, -0.039769649505615234, -0..."
1007177,1007177,"[0.02572963386774063, -0.040419720113277435, -..."
1007178,1007178,"[0.024163803085684776, -0.09511740505695343, -..."
...,...,...
1058124,1058124,"[0.025300081819295883, -0.06294917315244675, -..."
1058125,1058125,"[0.03510252758860588, -0.06641663610935211, -0..."
1058126,1058126,"[0.019733581691980362, -0.03408155217766762, -..."
1058127,1058127,"[0.01968446560204029, -0.017883658409118652, 0..."


In [ ]:
----- old method ----

In [13]:
_tmp_df.to_pickle("/mnt/vdb/phase3/TaiwaneseOolong.len20.100000_200000.pkl")

In [12]:
_tmp_df = df1
_tmp_df.drop(columns=['Sequence', 'length'], inplace=True)
_tmp_df['reps']=_h_avg.tolist()
_tmp_df

/home/ubuntu/miniconda3/envs/py3/lib/python3.7/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/ubuntu/miniconda3/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,ID,reps
100127,100127,"[0.023528747260570526, -0.03029148280620575, -..."
100128,100128,"[0.02823188528418541, -0.03292207047343254, -0..."
100129,100129,"[0.021869858726859093, -0.04351050406694412, -..."
100130,100130,"[0.020454389974474907, -0.041200652718544006, ..."
100131,100131,"[0.029424959793686867, -0.0632018893957138, -0..."
...,...,...
200357,200357,"[0.024071287363767624, -0.00011292310227872804..."
200358,200358,"[0.022646920755505562, -0.03028733655810356, -..."
200359,200359,"[0.02165237069129944, -0.0707235261797905, 0.0..."
200360,200360,"[0.019866550341248512, -0.02926749922335148, -..."
